# 实现过程中遇到的一些问题

## 已解决问题

### 训练的时候，各项损失都存在，但是正则项损失为 NaN

可能的原因有：
1. 检查学习率步长、正则项损失项的权重系数；
1. 网络中用到BatchNorm层，但是在训练一开始时，没有设置为训练阶段 keras.backend.set_learning_phase(True)；
1. 检查图片：数据如果进行了预处理，检查是否有可能导致输入为NaN，例如空白图在除以方差进行归一化，则有可能NaN；
1. 检查标签数据：本来应该为正的标签，出现了负数（如bounding box标注点反了，导致宽/高为负数）。


### model.fit函数打印的总损失，不等于损失计算过程中记录的各项损失之和

我在`yolov3/yolov3_loss.py`的`YOLOv3Loss`类中定义了6个`tf.Variable(shape=(3,))`，分别对应了各个head的损失函数中 **前期校正损失项（rectified_coord_loss）, 中心点坐标损失项（coord_loss_xy）, 预测宽高损失项（coord_loss_wh）, 背景损失项（noobj_iou_loss）, 前景损失项（obj_iou_loss）, 类别损失项（class_loss）**。然后在`utils/logger_callback.py`中调用了`self.model.losses`得到了模型的L2正则项。

如果日志打印中，总的损失函数值并不等于这些项的加和，有时大了，有时小了，因为总损失项做了一个历史平均的操作，而我打印的是当前迭代的子损失项的值。

## 遗留问题

### 加入条件分支获取校正损失项后，出现奇怪的错误

一开始实现前期计算anchor校正损失项时，我使用的是以下的实现代码逻辑（用迭代次数为条件判断是否需要计算校正损失项，如下代码`# 1`），但是不知道为什么一直报错。而如果把`# 1`的代码改为`# 2`处的代码，则不会报错。具体为什么我也没细究。

```python
def loss(self, targets, predicts):
    # 0. train-from-scratch 前期（12800 pic）坐标回归时，将预测wh回归为anchor，预测xy回归为grid的中心点
    def get_rectified_coord_loss():
        nonlocal predicts
        with tf.control_dependencies([keras.backend.update_add(self.current_num, tf.shape(predicts)[0])]):
            predicts = tf.reshape(predicts, shape=[-1, self.height, self.width, self.box_num, self.box_len])
            rectified_loss = tf.reduce_sum(tf.square(predicts[:, :, :, :, 0:4]), axis=[1, 2, 3, 4])
            rectified_loss = tf.multiply(self.rectified_loss_weight, rectified_loss)
            rectified_loss = tf.reduce_mean(rectified_loss, keepdims=True)
        return rectified_loss
    # 前期矫正的图片数小于预定义的坐标校正图片数，则继续加坐标校正损失
    rectified_coord_loss = tf.cond(self.current_num <= self.rectified_coord_num,
                                   lambda: get_rectified_coord_loss(),
                                   lambda: tf.zeros(shape=(1,), dtype=tf.float32))  # 1
    # rectified_coord_loss = get_rectified_coord_loss()  # 2
    # 1. 解码网络的输出
    decode_predicts, predicts_boxes = self.decoder.decode(predicts)
    # 2. 解码标签：(N, obj_num, gt_label_len)的标签矩阵, (N, obj_num, 4)的bounding boxes坐标
    targets, targets_boxes = self._decode_target(targets)
    # 3. 逐张图片计算损失函数，(N, 4)，按样本维度遍历
    yolov2_loss = tf.map_fn(lambda inp: self._single_image_loss(inp[0], inp[1], inp[2], inp[3]),
                            (targets, targets_boxes, decode_predicts, predicts_boxes), dtype=tf.float32,
                            parallel_iterations=1)
    yolov2_loss = tf.reduce_mean(yolov2_loss, axis=0)
    # 4. 汇总并记录所有损失 (6,)
    update_op = [
        self.coord_loss_xy.assign(yolov2_loss[0]),
        self.coord_loss_wh.assign(yolov2_loss[1]),
        self.noobj_iou_loss.assign(yolov2_loss[2]),
        self.obj_iou_loss.assign(yolov2_loss[3]),
        self.class_loss.assign(yolov2_loss[4]),
        self.rectified_coord_loss.assign(rectified_coord_loss)
    ]
    # 前期矫正的图片数小于预定义的坐标校正图片数，则继续加坐标校正损失
    total_loss = tf.concat([yolov2_loss, rectified_coord_loss], axis=-1)
    # 4. 汇总所有损失
    with tf.control_dependencies(update_op):
        total_loss = tf.reduce_sum(total_loss)
    return total_loss
```

后面我直接把条件判断转移到`4 损失汇总`去了，然后也不会再报错：

```python
# 前期矫正的图片数小于预定义的坐标校正图片数，则继续加坐标校正损失
# [rectified_coord_loss, coord_loss_xy, coord_loss_wh, noobj_iou_loss, obj_iou_loss, class_loss]
total_loss = tf.cond(self.current_num <= self.rectified_coord_num,
                     lambda: tf.concat([yolov2_loss, get_rectified_coord_loss()], axis=-1),
                     lambda: yolov2_loss)
```